In [27]:
import pandas as pd
import numpy as np
import os

def compute_fixation_stats(file_path, angle_threshold):
    
# Load OpenFace eye-tracking data (CSV file)
    data = pd.read_csv(file_path, index_col = 0, delimiter=',')  # Replace 'openface_eye_tracking_data.csv' with your file path
    data.columns = data.columns.str.strip()
    # print(data.columns)

    # Extract relevant columns from the dataset
    time = data['unix']  # Timestamps
    # time_intervals=np.diff(time)
    gaze_x = data['gaze_0_x']  # Gaze direction in X-axis
    gaze_y = data['gaze_0_y']  # Gaze direction in Y-axis
    gaze_angle_x = np.degrees(data['gaze_angle_x'])  # Gaze direction in X-axis
    gaze_angle_y = np.degrees(data['gaze_angle_y'])  # Gaze direction in Y-axis
    velocity_x = np.diff(gaze_angle_x)/np.diff(time)
    velocity_y = np.diff(gaze_angle_y)/np.diff(time)
    velocity = np.sqrt(velocity_x**2 + velocity_y**2)
    # acceleration_x = np.diff(velocity_x)/np.diff(time)
    # acceleration_y = np.gradient(velocity_y)/np.diff(time)
    # acceleration = np.sqrt(acceleration_x**2 + acceleration_y**2)
    distances = np.sqrt(np.diff(gaze_angle_x)**2 + np.diff(gaze_angle_y)**2)
    total_path_length=np.sum(distances)
   

    fixation_start = []
    fixation_end = []
    is_fixating = False
    for index in range(1, len(data)):
        # Check if the difference exceeds the threshold
        if ((abs(gaze_angle_x.iloc[index] - gaze_angle_x.iloc[index - 1]) +
                abs(gaze_angle_y.iloc[index] - gaze_angle_y.iloc[index - 1])) < angle_threshold):
            if not is_fixating:
                fixation_start.append(index)
                is_fixating = True
        else:
            if is_fixating:
                fixation_end.append(index - 1)
                is_fixating = False

    # If fixation was ongoing until the end of the data, add the last index as end point
    if is_fixating:
        fixation_end.append(len(data) - 1)
         
     # Calculate fixation durations
    fixation_durations=[]
    for start, end in zip(fixation_start, fixation_end):
        duration = time.iloc[end] - time.iloc[start]
        fixation_durations.append(duration)
        

    # Calculate average gaze direction during fixations
    average_gaze_x = [gaze_x.loc[start:end].mean() for start, end in zip(fixation_start, fixation_end)]
    average_gaze_y = [gaze_y.loc[start:end].mean() for start, end in zip(fixation_start, fixation_end)]
    average_velocity = [np.mean(velocity[start:end+1]) for start, end in zip(fixation_start, fixation_end)]
    # average_acceleration = [np.mean(acceleration[start:end+1]) for start, end in zip(fixation_start, fixation_end)]


    results_df = pd.DataFrame({
    'Fixation_Start_Time': [time.iloc[start] for start, end, duration in zip(fixation_start, fixation_end, fixation_durations)
                            if duration >= 100],
    'Fixation_End_Time': [time.iloc[end] for start, end, duration in zip(fixation_start, fixation_end, fixation_durations)
                          if duration >= 100],
    'Fixation_Duration': [duration for start, end, duration in zip(fixation_start, fixation_end, fixation_durations)
                          if duration >= 100],
    'Average_Gaze_X': [avg_x for duration, avg_x in zip(fixation_durations, average_gaze_x) if duration >= 100],
    'Average_Gaze_Y': [avg_y for duration, avg_y in zip(fixation_durations, average_gaze_y) if duration >= 100],
    'Average_Velocity': [vel for duration, vel in zip(fixation_durations, average_velocity) if duration >= 100],
    # 'Average_Acceleration': [acc for duration, acc in zip(fixation_durations, average_acceleration) if duration >= 100]
    
    })

    # Save the results to a new CSV file
    folder_path, file_name = os.path.split(file_path)
    output_file_name =folder_path+'/output_files/'+ f'{file_name}_fixation.csv'
    results_df.to_csv(output_file_name, index=False)  # Change 'eye_tracking_results.csv' to your desired file name
    # print('Results saved to eye_tracking_results.csv')

    # Compute additional statistics

    # Filter fixation durations less than 100 ms
    filtered_fixation_durations = [duration for duration in fixation_durations if duration >= 100]
    filtered_average_velocity = [avg_vel for avg_vel, duration in zip(average_velocity, fixation_durations) if duration >= 100]
    
    # Compute additional statistics for valid fixations
    mean_fixation_duration = np.mean(filtered_fixation_durations)
    median_fixation_duration = np.median(filtered_fixation_durations)
    std_fixation_duration = np.std(filtered_fixation_durations)
    min_fixation_duration = np.min(filtered_fixation_durations)
    max_fixation_duration = np.max(filtered_fixation_durations)
    mean_average_velocity = np.mean(filtered_average_velocity)
    median_average_velocity = np.median(filtered_average_velocity)
    std_average_velocity = np.std(filtered_average_velocity)
    min_average_velocity = np.min(filtered_average_velocity)
    max_average_velocity = np.max(filtered_average_velocity)
   
      # Return fixation statistics
    return {
        'File_ID': os.path.splitext(os.path.basename(file_path))[0],
        'Mean_fixation_duration': mean_fixation_duration,
        'Median_fixation_duration': median_fixation_duration,
        'Standard_Deviation_fixation_duration': std_fixation_duration,
        'Minimum_fixation_duration': min_fixation_duration,
        'Maximum_fixation_duration': max_fixation_duration,
        'Mean_fixation_velocity': mean_average_velocity,
        'Median_fixation_velocity': median_average_velocity,
        'Standard_Deviation_fixation_velocity': std_average_velocity,
        'Minimum_fixation_velocity': min_average_velocity,
        'Maximum_fixation_velocity': max_average_velocity,
        'Total_path_length':total_path_length
    }
    
def detect_saccades(file_path, angle_threshold,velocity_threshold, acceleration_threshold):
    """
    Detects saccades in the OpenFace dataset and computes statistics.

    źarameters:
        file_path (str): Path to the OpenFace dataset CSV file.
        velocity_threshold (float): Threshold for velocity of gaze direction changes.
        acceleration_threshold (float): Threshold for acceleration of gaze direction changes.

    Returns:
        pd.DataFrame: DataFrame containing saccade data with timestamps and indices.
        dict: Dictionary containing statistics for the detected saccades.
    """
    # Load OpenFace CSV file containing facial landmark data
    openface_data = pd.read_csv(file_path, index_col = 0, delimiter=',')  # Replace 'openface_eye_tracking_data.csv' with your file path
    openface_data.columns = openface_data.columns.str.strip()
    
    # Extract timestamps
    timestamps = openface_data['unix']
    time_intervals=np.diff(timestamps)
    
    # gaze_x = openface_data['gaze_0_x']  # Gaze direction in X-axis
    # gaze_y = openface_data['gaze_0_y']  # Gaze direction in Y-axis

    # Extract gaze direction angles
    gaze_angle_x =openface_data['gaze_angle_x']
    gaze_angle_y =openface_data['gaze_angle_y']

# Compute changes in gaze direction
    gaze_angle_x_diff = np.diff(gaze_angle_x)
    gaze_angle_y_diff = np.diff(gaze_angle_y)
    gaze_angle_x_changes=np.abs(gaze_angle_x_diff)
    gaze_angle_y_changes=np.abs(gaze_angle_y_diff)
  # Compute velocity of gaze direction changes
    velocity_x = np.diff(gaze_angle_x)/time_intervals
    velocity_y = np.diff(gaze_angle_y)/time_intervals
    velocity = np.sqrt(velocity_x**2 + velocity_y**2)
    distances = np.sqrt(np.diff(gaze_angle_x)**2 + np.diff(gaze_angle_y)**2)
  
    # Compute acceleration of gaze direction changes
    # acceleration_x = np.diff(velocity_x)/time_intervals
    # acceleration_y = np.diff(velocity_y)/time_intervals
    # acceleration = np.sqrt(acceleration_x**2 + acceleration_y**2)

    # Convert velocity and acceleration thresholds to radians per second and radians per second squared
    velocity_threshold_rad_per_sec = np.deg2rad(velocity_threshold)
    acceleration_threshold_rad_per_sec_sq = np.deg2rad(acceleration_threshold)
    # print(velocity_threshold_rad_per_sec)

    # Detect saccades based on thresholds
    # saccade_indices = np.where(
    #     (velocity > velocity_threshold_rad_per_sec)    
    # )[0]
    
    # combined_angle_changes = np.sqrt(gaze_angle_x_changes**2 + gaze_angle_y_changes**2)
    # saccade_indices = np.where(combined_angle_changes > angle_threshold)[0] + 1
    # # Classify saccades based on duration and velocity thresholds
    # saccade_durations = np.diff(timestamps.iloc[saccade_indices])
    
    saccade_start = []
    saccade_end = []
    is_saccading = False
    saccade_indices=[]

    # Loop through the data starting from the second index
    for index in range(1, len(openface_data)):
        # Calculate combined change in gaze angles (x and y)
        combined_angle_change = abs(gaze_angle_x.iloc[index] - gaze_angle_x.iloc[index - 1]) + abs(gaze_angle_y.iloc[index] - gaze_angle_y.iloc[index - 1])
        
        # Check if the combined change exceeds the threshold
        if combined_angle_change > angle_threshold:
            saccade_indices.append(index)
            if not is_saccading:
                saccade_start.append(index)
            is_saccading = True
        else:
            if is_saccading:
                saccade_end.append(index - 1)
            is_saccading = False

    # If a saccade was ongoing until the end of the data, add the last index as end point
    if is_saccading:
        saccade_end.append(len(openface_data) - 1)
        
    # Calculate saccade durations
    saccade_durations = []
    for start, end in zip(saccade_start, saccade_end):
        duration = timestamps.iloc[end] - timestamps.iloc[start]
        saccade_durations.append(duration) 
        
    average_velocity = [np.mean(velocity[start:end+1]) for start, end in zip(saccade_start, saccade_end)]
    min_velocity = [np.min(velocity[start:end+1]) for start, end in zip(saccade_start, saccade_end)]
    max_velocity = [np.max(velocity[start:end+1]) for start, end in zip(saccade_start, saccade_end)]
    path_length=[np.sum(distances[start:end+1]) for start, end in zip(saccade_start, saccade_end)]
    
    saccade_types = []
    for duration in saccade_durations:
        if duration < 20 and duration:
            saccade_types.append('Micro')
        elif duration >= 20 and duration < 80:
            saccade_types.append('Standard')
        elif duration >= 80 and duration <200:
            saccade_types.append('Long')
        else:
            saccade_types.append('fixation')
    #         # Ensure that saccade_types has the same length as saccade_indices
    # while len(saccade_types) < len(saccade_durations):
    #     saccade_types.append('fixation')
    

    # Save saccade data to a DataFrame
    saccade_df = pd.DataFrame({
        # 'Timestamp': timestamps.iloc[saccade_indices],
        # 'Saccade_Index': saccade_indices,
        'Saccade_Start': saccade_start,
        'Saccade_End': saccade_end,
        'Saccade_Type': saccade_types,
        'Saccade_Duration': saccade_durations,
        'Saccade_Mean_Velocity':average_velocity,
        'Saccade_Min_Velocity':min_velocity,
        'Saccade_Max_Velocity':max_velocity,
        'Saccade_Path_Length:': path_length,
            
    })
    
    saccade_data=saccade_df[saccade_df['Saccade_Type'] != 'fixation']
    

    # Save the saccade data to a CSV file
    folder_path, file_name = os.path.split(file_path)
    output_file_name = folder_path + '/output_files/' + f'{file_name}_saccade_data.csv'
    saccade_data.to_csv(output_file_name, index=False)
    
    saccade_stats = {}
    # for saccade_type in set(saccade_data['Saccade_Type']):
    #     type_indices = saccade_data[saccade_data['Saccade_Type'] == saccade_type].index
    #     type_durations = [saccade_durations[i] for i in type_indices]  # Ensure valid indices
    #     if type_durations:
    #         mean_duration = np.mean(type_durations)
    #         median_duration = np.median(type_durations)
    #         std_duration = np.std(type_durations)
    #         min_duration = np.min(type_durations)
    #         max_duration = np.max(type_durations)

    #         saccade_stats[f'Mean_Duration_{saccade_type}'] = mean_duration
    #         saccade_stats[f'Median_Duration_{saccade_type}'] = median_duration
    #         saccade_stats[f'Std_Duration_{saccade_type}'] = std_duration
    #         saccade_stats[f'Min_Duration_{saccade_type}'] = min_duration
    #         saccade_stats[f'Max_Duration_{saccade_type}'] = max_duration
    if saccade_data.empty:
        saccade_stats = {
            'File_ID': os.path.splitext(os.path.basename(file_path))[0],
            'Total_Micro': 0,
            'Total_Standard': 0,
            'Total_Long': 0,
            'Total_All': 0,
            'Max_Duration': np.nan,
            'Min_Duration': np.nan,
            'Mean_Duration': np.nan,
            'Peak_Velocity': np.nan,
            'Mean_Peak_Velocity': np.nan,
            'Mean_Velocity': np.nan,
            'Total_Path_Length': 0
        }
    else:

        # Compute total counts for each saccade type
        total_micro = saccade_data['Saccade_Type'].value_counts().get('Micro', 0)
        total_standard = saccade_data['Saccade_Type'].value_counts().get('Standard', 0)
        total_long = saccade_data['Saccade_Type'].value_counts().get('Long', 0)
        total_all = len(saccade_data)
        mean_duration= saccade_data['Saccade_Duration'].mean()
        max_duration= saccade_data['Saccade_Duration'].max()
        min_duration= saccade_data['Saccade_Duration'].min()

        # Save saccade statistics to the dictionary
        saccade_stats.update({
            'File_ID': os.path.splitext(os.path.basename(file_path))[0],
            'Total_Micro': total_micro,
            'Total_Standard': total_standard,
            'Total_Long': total_long,
            'Total_All': total_all,
            'Max_Duration': max_duration,
            'Min_Duration': min_duration,
            'Mean_Duration':mean_duration,
            'Peak_Velocity': np.max(max_velocity),
            'Mean_Peak_Velocity': np.mean(max_velocity),
            'Mean_Velocity':np.mean(average_velocity),
            'Total_Path_Length': np.sum(path_length)
        })

    return saccade_stats


In [28]:

# Directory containing CSV files
directory = '/Users/openai-projects/openai-guidelines/openFace_files/of_files'

# # Threshold for fixation detection
# threshold = 200  # Adjust as needed

# List to store computed statistics for all files
stats_fixation_list = []
stats_saccade_list = []

# Loop through each CSV file in the directory
for file_name in os.listdir(directory):
    if file_name.endswith('.csv'):
        file_path = os.path.join(directory, file_name)
        # Compute fixation statistics for the current file
        stats = compute_fixation_stats(file_path, angle_threshold=1)
        stats_saccade = detect_saccades(file_path, angle_threshold=0.080, velocity_threshold=5, acceleration_threshold=50)
        
        # Append statistics to the list
        stats_fixation_list.append(stats)
        stats_saccade_list.append(stats_saccade)

# Create a DataFrame from the list of statistics
stats_fixation_df = pd.DataFrame(stats_fixation_list)
stats_saccade_df = pd.DataFrame(stats_saccade_list)

# Modify the output file name with a suffix
output_file_name_1 = 'fixation_statistics.csv'
output_file_name_2 = 'saccade_statistics.csv'

# Save the statistics to a CSV file with the modified name
stats_fixation_df.to_csv(output_file_name_1, index=False)
stats_saccade_df.to_csv(output_file_name_2, index=False)

# print('Fixation statistics saved to', output_file_name_1)

/Users/openai-projects/openai-guidelines/openai-env/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/openai-projects/openai-guidelines/openai-env/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/kc/596t6rmx1dn7bfyv5hfgmqm00000gn/T/ipykernel_900/3559044075.py:8: DtypeWarning: Columns (43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,